In [ ]:
# Le test pour testDepartureStopsInforToulouse.gaml
# GTFS Toulouse - Analyse des arrêts de départ
# Il affiche les arrêts de départ, le nombre de trips associés, et simule la logique de bus similaire à GAMA.
# Assurez-vous que les fichiers GTFS sont correctement placés dans le dossier spécifié.
import os
import csv

gtfs_dir = r"C:\Users\tiend\Downloads\Toulouse_GTFS"
stop_times_file = os.path.join(gtfs_dir, "stop_times.txt")
calendar_dates_file = os.path.join(gtfs_dir, "calendar_dates.txt")
trips_file = os.path.join(gtfs_dir, "trips.txt")

def detect_separator(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        sample = f.readline()
        for sep in [',', ';', '\t']:
            if sep in sample:
                return sep
    return ','

target_date = "20250610"   # <-- à modifier selon la date voulue

# 1. Lire les service_id actifs pour cette date
active_service_ids = set()
if os.path.exists(calendar_dates_file):
    sep = detect_separator(calendar_dates_file)
    with open(calendar_dates_file, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f, delimiter=sep)
        for row in reader:
            if row["date"] == target_date and row["exception_type"] == "1":
                active_service_ids.add(row["service_id"])
else:
    print("Fichier calendar_dates.txt non trouvé.")
    exit(1)

# 2. Lire les trip_id correspondant à ces service_id dans trips.txt
active_trip_ids = set()
if os.path.exists(trips_file):
    sep = detect_separator(trips_file)
    with open(trips_file, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f, delimiter=sep)
        for row in reader:
            if row["service_id"] in active_service_ids:
                active_trip_ids.add(row["trip_id"])
else:
    print("Fichier trips.txt non trouvé.")
    exit(1)

# 3. Même logique que précédemment sur stop_times.txt mais seulement avec ces trip_id
departure_stops = set()       # stop_id
trips_with_departure = set()  # trip_id associé à un stop départ
trips_all_stops_count = {}    # trip_id -> nombre de stops (tous trips)

if os.path.exists(stop_times_file):
    sep = detect_separator(stop_times_file)
    with open(stop_times_file, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f, delimiter=sep)
        for row in reader:
            trip_id = row["trip_id"]
            if trip_id not in active_trip_ids:
                continue

            stop_id = row["stop_id"]
            stop_sequence = row["stop_sequence"]

            if trip_id not in trips_all_stops_count:
                trips_all_stops_count[trip_id] = 0
            trips_all_stops_count[trip_id] += 1

            if stop_sequence.strip() == "1":
                departure_stops.add(stop_id)
                trips_with_departure.add(trip_id)

    n_departure_stops = len(departure_stops)
    n_trips_departure = len(trips_with_departure)
    total_stops_in_departure_trips = sum(trips_all_stops_count[trip_id] for trip_id in trips_with_departure)

    print(f"==== Analyse des stops de départ (GTFS) pour le {target_date} ====")
    print(f"1. Nombre de stops de départ (stop_sequence==1, distincts)           : {n_departure_stops}")
    print(f"2. Nombre de trips associés à ces stops de départ                     : {n_trips_departure}")
    print(f"3. Nombre total de stops (tous trips concernés)                       : {total_stops_in_departure_trips}")
else:
    print("Fichier stop_times.txt non trouvé.")


==== Analyse des stops de départ (GTFS) pour le 20250610 ====
1. Nombre de stops de départ (stop_sequence==1, distincts)           : 232
2. Nombre de trips associés à ces stops de départ                     : 12226
3. Nombre total de stops (tous trips concernés)                       : 272541


In [ ]:
#  Le test pour testDepartureStopsInforHanoi.gaml
# GTFS Hanoi - Analyse des arrêts de départ
# Il affiche les arrêts de départ, le nombre de trips associés, et simule la logique de bus similaire à GAMA.
# Assurez-vous que les fichiers GTFS sont correctement placés dans le dossier spécifié.
import os
import csv
from collections import defaultdict
import datetime

# Configuration pour Hanoi GTFS
gtfs_dir = r"C:\Users\tiend\Downloads\GTFS_Hanoi"
stop_times_file = os.path.join(gtfs_dir, "stop_times.txt")
calendar_file = os.path.join(gtfs_dir, "calendar.txt")
trips_file = os.path.join(gtfs_dir, "trips.txt")
stops_file = os.path.join(gtfs_dir, "stops.txt")
shapes_file = os.path.join(gtfs_dir, "shapes.txt")

DATE_CIBLE = "20180101"  # 1er janvier 2018

def detect_separator(file_path):
    """Détecte le séparateur utilisé dans le fichier CSV"""
    with open(file_path, 'r', encoding='utf-8') as f:
        sample = f.readline()
        for sep in [',', ';', '\t']:
            if sep in sample:
                return sep
    return ','

def load_csv_data(file_path, key_field=None):
    """Charge les données CSV dans un dictionnaire ou une liste"""
    if not os.path.exists(file_path):
        print(f"❌ Fichier {file_path} non trouvé.")
        return {} if key_field else []
    
    sep = detect_separator(file_path)
    data = {} if key_field else []
    
    with open(file_path, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f, delimiter=sep)
        for row in reader:
            # Nettoyer les espaces dans les clés et valeurs
            clean_row = {k.strip(): v.strip() if v else v for k, v in row.items()}
            
            if key_field:
                key = clean_row.get(key_field)
                if key:
                    data[key] = clean_row
            else:
                data.append(clean_row)
    
    return data

def get_day_of_week(date_str):
    """Convertit une date YYYYMMDD en jour de la semaine"""
    try:
        year = int(date_str[:4])
        month = int(date_str[4:6])
        day = int(date_str[6:8])
        date_obj = datetime.date(year, month, day)
        
        # Convertir en format GTFS (lundi=1, dimanche=0)
        weekday = date_obj.weekday()  # Python: lundi=0, dimanche=6
        gtfs_weekday = (weekday + 1) % 7  # GTFS: lundi=1, dimanche=0
        
        days = ['sunday', 'monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday']
        return days[gtfs_weekday]
    except:
        print(f"❌ Erreur lors du parsing de la date: {date_str}")
        return None

def date_in_range(date_str, start_date, end_date):
    """Vérifie si une date est dans la plage donnée"""
    try:
        return start_date <= date_str <= end_date
    except:
        return False

def get_active_service_ids_from_calendar(date_cible):
    """Étape 1: Récupération des service_id actifs depuis calendar.txt"""
    print(f"🔍 Recherche des services actifs pour le {date_cible} dans calendar.txt...")
    
    # Déterminer le jour de la semaine
    day_of_week = get_day_of_week(date_cible)
    if not day_of_week:
        return set()
    
    print(f"📅 Le {date_cible} est un {day_of_week}")
    
    calendar_data = load_csv_data(calendar_file)
    active_service_ids = set()
    
    for row in calendar_data:
        service_id = row.get("service_id")
        start_date = row.get("start_date")
        end_date = row.get("end_date")
        day_active = row.get(day_of_week)
        
        if (service_id and start_date and end_date and day_active and
            day_active == "1" and date_in_range(date_cible, start_date, end_date)):
            active_service_ids.add(service_id)
            print(f"   ✅ Service {service_id} actif ({start_date} → {end_date})")
    
    print(f"✅ {len(active_service_ids)} services actifs trouvés pour {day_of_week}")
    return active_service_ids

def get_active_trips(active_service_ids):
    """Étape 2: Récupération des trips associés aux services actifs"""
    print(f"🔍 Recherche des trips pour {len(active_service_ids)} services...")
    
    trips_data = load_csv_data(trips_file, "trip_id")
    active_trips = {}
    
    for trip_id, trip_data in trips_data.items():
        service_id = trip_data.get("service_id")
        if service_id in active_service_ids:
            active_trips[trip_id] = trip_data
    
    print(f"✅ {len(active_trips)} trips actifs trouvés")
    return active_trips

def load_stops_data():
    """Chargement des données des arrêts"""
    print("🔍 Chargement des données des arrêts...")
    stops_data = load_csv_data(stops_file, "stop_id")
    print(f"✅ {len(stops_data)} arrêts chargés")
    return stops_data

def load_shapes_data():
    """Chargement des données des formes/trajets"""
    print("🔍 Chargement des données des shapes...")
    shapes_data = defaultdict(list)
    
    if os.path.exists(shapes_file):
        raw_shapes = load_csv_data(shapes_file)
        for row in raw_shapes:
            shape_id = row.get("shape_id")
            if shape_id:
                try:
                    shapes_data[shape_id].append({
                        'lat': float(row.get("shape_pt_lat", 0)),
                        'lon': float(row.get("shape_pt_lon", 0)),
                        'sequence': int(row.get("shape_pt_sequence", 0))
                    })
                except ValueError:
                    continue
        
        # Trier par séquence
        for shape_id in shapes_data:
            shapes_data[shape_id].sort(key=lambda x: x['sequence'])
    else:
        print("⚠️ Fichier shapes.txt non trouvé")
    
    print(f"✅ {len(shapes_data)} shapes chargés")
    return shapes_data

def analyze_stop_times(active_trips, stops_data):
    """Étape 3: Analyse des stop_times (équivalent à l'analyse GAMA)"""
    print("🔍 Analyse des stop_times...")
    
    # Structures de données équivalentes à GAMA
    trip_shape_map = {}  # trip_id -> shape_id
    departure_stops_info = defaultdict(dict)  # stop_id -> {trip_id: [(stop, time), ...]}
    all_stop_times = defaultdict(list)  # trip_id -> [(stop_data, time), ...]
    
    # Statistiques
    departure_stops = set()
    trips_with_departure = set()
    trips_all_stops_count = defaultdict(int)
    
    if not os.path.exists(stop_times_file):
        print("❌ Fichier stop_times.txt non trouvé.")
        return {}, {}, {}
    
    sep = detect_separator(stop_times_file)
    with open(stop_times_file, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f, delimiter=sep)
        
        for row in reader:
            trip_id = row.get("trip_id", "").strip()
            stop_id = row.get("stop_id", "").strip()
            stop_sequence = row.get("stop_sequence", "").strip()
            departure_time = row.get("departure_time", "").strip()
            
            if trip_id in active_trips:
                # Comptage pour statistiques
                trips_all_stops_count[trip_id] += 1
                
                # Récupération des données de l'arrêt
                stop_data = stops_data.get(stop_id, {})
                stop_info = {
                    'stop_id': stop_id,
                    'stop_name': stop_data.get('stop_name', 'Inconnu'),
                    'stop_lat': float(stop_data.get('stop_lat', 0)) if stop_data.get('stop_lat') else 0,
                    'stop_lon': float(stop_data.get('stop_lon', 0)) if stop_data.get('stop_lon') else 0,
                    'sequence': int(stop_sequence) if stop_sequence.isdigit() else 0
                }
                
                # Ajout à la liste des stop_times pour ce trip
                all_stop_times[trip_id].append((stop_info, departure_time))
                
                # Gestion des arrêts de départ (stop_sequence == 1)
                if stop_sequence == "1":
                    departure_stops.add(stop_id)
                    trips_with_departure.add(trip_id)
                    
                    # Récupération du shape_id depuis trips_data
                    trip_data = active_trips.get(trip_id, {})
                    shape_id = trip_data.get("shape_id")
                    if shape_id:
                        trip_shape_map[trip_id] = shape_id
    
    # Trier les stop_times par séquence pour chaque trip
    for trip_id in all_stop_times:
        all_stop_times[trip_id].sort(key=lambda x: x[0]['sequence'])
        
        # Construction de departure_stops_info pour l'arrêt de départ
        if trip_id in trips_with_departure and all_stop_times[trip_id]:
            first_stop = all_stop_times[trip_id][0][0]
            first_stop_id = first_stop['stop_id']
            departure_stops_info[first_stop_id][trip_id] = all_stop_times[trip_id]
    
    # Statistiques finales
    n_departure_stops = len(departure_stops)
    n_trips_departure = len(trips_with_departure)
    total_stops_in_departure_trips = sum(trips_all_stops_count[trip_id] for trip_id in trips_with_departure)
    
    print("=" * 80)
    print(f"📊 ANALYSE DES DONNÉES GTFS HANOI POUR LE {DATE_CIBLE}")
    print("=" * 80)
    print(f"1. Nombre de stops de départ (stop_sequence==1, distincts)    : {n_departure_stops}")
    print(f"2. Nombre de trips associés à ces stops de départ            : {n_trips_departure}")
    print(f"3. Nombre total de stops (tous trips concernés)              : {total_stops_in_departure_trips}")
    print(f"4. Nombre de trips actifs au total                           : {len(active_trips)}")
    print("=" * 80)
    
    return trip_shape_map, departure_stops_info, all_stop_times

def find_available_trip_and_stop(trip_shape_map, departure_stops_info, stops_data):
    """Trouve un trip et un stop disponibles pour l'analyse"""
    print(f"\n🔍 RECHERCHE D'UN TRIP ET STOP DISPONIBLES")
    print("-" * 80)
    
    # Lister quelques trips disponibles
    available_trips = list(trip_shape_map.keys())[:10]
    print(f"📋 Premiers trips disponibles: {available_trips}")
    
    

def analyze_specific_trip_and_stop(trip_id, stop_id, trip_shape_map, departure_stops_info, stops_data):
    """Analyse spécifique du trip et de l'arrêt sélectionnés"""
    print(f"\n🎯 ANALYSE SPÉCIFIQUE - Trip: {trip_id}, Stop: {stop_id}")
    print("-" * 80)
    
    if not trip_id or not stop_id:
        print("❌ Trip ou stop invalide")
        return None
    
    selected_stop_data = stops_data.get(stop_id, {})
    print(f"✅ Arrêt sélectionné: {selected_stop_data.get('stop_name', 'Inconnu')} (ID: {stop_id})")
    
    # Vérification du trip_id dans trip_shape_map
    if trip_id in trip_shape_map:
        shape_id = trip_shape_map[trip_id]
        print(f"✅ Shape ID pour le trip {trip_id}: {shape_id}")
    else:
        print(f"❌ Trip {trip_id} non trouvé dans trip_shape_map")
        return None
    
    # Recherche des stops pour ce trip (peut être dans n'importe quel departure stop)
    stops_for_trip = None
    for departure_stop_id, trips_dict in departure_stops_info.items():
        if trip_id in trips_dict:
            stops_for_trip = trips_dict[trip_id]
            print(f"✅ Trip trouvé depuis l'arrêt de départ: {departure_stop_id}")
            break
    
    if stops_for_trip:
        print(f"✅ Nombre d'arrêts pour le trip {trip_id}: {len(stops_for_trip)}")
        
        # Affichage des arrêts du trip
        print("\n📍 Liste des arrêts du trip:")
        for i, (stop_info, time) in enumerate(stops_for_trip):
            print(f"   {i+1:2d}. {stop_info['stop_name']} (séq: {stop_info['sequence']}) - {time}")
        
        return stops_for_trip
    else:
        print(f"❌ Aucun arrêt trouvé pour le trip {trip_id}")
        return None

def simulate_bus_logic(stops_for_trip, shapes_data, shape_id):
    """Simulation de la logique du bus (équivalent GAMA)"""
    if not stops_for_trip:
        print("❌ Pas de données d'arrêts pour simuler le bus")
        return
    
    print(f"\n🚌 SIMULATION DU BUS")
    print("-" * 50)
    
    # Filtrage des arrêts valides (équivalent au filtrage NIL de GAMA)
    valid_stops = []
    for stop_info, time in stops_for_trip:
        if stop_info and stop_info.get('stop_lat') and stop_info.get('stop_lon'):
            valid_stops.append((stop_info, time))
    
    print(f"✅ {len(valid_stops)} arrêts valides sur {len(stops_for_trip)} total")
    
    if len(valid_stops) < 2:
        print("❌ Moins de 2 arrêts valides - simulation impossible")
        return
    
    # Récupération des points du shape pour le snapping
    shape_points = shapes_data.get(shape_id, [])
    print(f"✅ Shape {shape_id} contient {len(shape_points)} points")
    
    # Simulation du snapping (logique simplifiée)
    snapped_locations = []
    for stop_info, time in valid_stops:
        stop_lat, stop_lon = stop_info['stop_lat'], stop_info['stop_lon']
        
        if shape_points:
            # Trouver le point du shape le plus proche (distance euclidienne simple)
            min_dist = float('inf')
            closest_point = None
            for point in shape_points:
                dist = ((stop_lat - point['lat'])**2 + (stop_lon - point['lon'])**2)**0.5
                if dist < min_dist:
                    min_dist = dist
                    closest_point = point
            snapped_locations.append(closest_point)
        else:
            snapped_locations.append({'lat': stop_lat, 'lon': stop_lon})
    
    print(f"✅ {len(snapped_locations)} positions snappées calculées")
    
    # Simulation du parcours du bus
    print("\n🚌 Parcours simulé du bus:")
    for i, ((stop_info, time), snapped_pos) in enumerate(zip(valid_stops, snapped_locations)):
        status = "🚌 DÉPART" if i == 0 else "🏁 ARRIVÉE" if i == len(valid_stops)-1 else "🚏 PASSAGE"
        print(f"   {i+1:2d}. {status} {stop_info['stop_name']} à {time}")
        print(f"       Position snappée: lat={snapped_pos['lat']:.6f}, lon={snapped_pos['lon']:.6f}")
    
    return valid_stops, snapped_locations

def main():
    """Fonction principale reproduisant la logique GAMA pour Hanoi"""
    print("🚀 DÉMARRAGE DE L'ANALYSE GTFS HANOI")
    print("=" * 80)
    
    # Étapes équivalentes à l'init GAMA
    active_service_ids = get_active_service_ids_from_calendar(DATE_CIBLE)
    if not active_service_ids:
        print("❌ Aucun service actif trouvé pour cette date")
        return
    
    active_trips = get_active_trips(active_service_ids)
    if not active_trips:
        print("❌ Aucun trip actif trouvé")
        return
    
    stops_data = load_stops_data()
    shapes_data = load_shapes_data()
    
    trip_shape_map, departure_stops_info, all_stop_times = analyze_stop_times(active_trips, stops_data)
    
    print("\n✅ Analyse terminée")

if __name__ == "__main__":
    main()

🚀 DÉMARRAGE DE L'ANALYSE GTFS HANOI
🔍 Recherche des services actifs pour le 20180101 dans calendar.txt...
📅 Le 20180101 est un monday
   ✅ Service FULLW actif (20180101 → 20181231)
✅ 1 services actifs trouvés pour monday
🔍 Recherche des trips pour 1 services...
✅ 6713 trips actifs trouvés
🔍 Chargement des données des arrêts...
✅ 7670 arrêts chargés
🔍 Chargement des données des shapes...
⚠️ Fichier shapes.txt non trouvé
✅ 0 shapes chargés
🔍 Analyse des stop_times...
📊 ANALYSE DES DONNÉES GTFS HANOI POUR LE 20180101
1. Nombre de stops de départ (stop_sequence==1, distincts)    : 224
2. Nombre de trips associés à ces stops de départ            : 6713
3. Nombre total de stops (tous trips concernés)              : 224166
4. Nombre de trips actifs au total                           : 6713

✅ Analyse terminée


In [ ]:
#  Le test pour testDepartureStopsInforNantes.gaml
# GTFS Hanoi - Analyse des arrêts de départ
# Il affiche les arrêts de départ, le nombre de trips associés, et simule la logique de bus similaire à GAMA.
# Assurez-vous que les fichiers GTFS sont correctement placés dans le dossier spécifié.

import os
import csv
import datetime

gtfs_dir = r"C:\Users\tiend\Downloads\gtfs-tan"
stop_times_file = os.path.join(gtfs_dir, "stop_times.txt")
calendar_file = os.path.join(gtfs_dir, "calendar.txt")
trips_file = os.path.join(gtfs_dir, "trips.txt")

def detect_separator(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        sample = f.readline()
        for sep in [',', ';', '\t']:
            if sep in sample:
                return sep
    return ','

def convert_time_to_seconds(time_str):
    """Convertit une heure HH:MM:SS en secondes (comme dans le code Java)"""
    try:
        parts = time_str.split(":")
        hours = int(parts[0])
        minutes = int(parts[1])
        seconds = int(parts[2])
        return hours * 3600 + minutes * 60 + seconds
    except:
        return 0

target_date = "20250515"   # Date correspondant au modèle GAML

# 1. Service_ids actifs pour cette date (calendar.txt)
active_service_ids = set()
if os.path.exists(calendar_file):
    sep = detect_separator(calendar_file)
    with open(calendar_file, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f, delimiter=sep)
        for row in reader:
            if row["start_date"] <= target_date <= row["end_date"]:
                weekday = datetime.datetime.strptime(target_date, "%Y%m%d").weekday()
                week_col = ["monday","tuesday","wednesday","thursday","friday","saturday","sunday"][weekday]
                if row[week_col] == "1":
                    active_service_ids.add(row["service_id"])
else:
    print("Fichier calendar.txt non trouvé.")
    exit(1)

print(f"Services actifs trouvés: {len(active_service_ids)}")

# 2. Trip_ids actifs pour ces service_id
active_trip_ids = set()
if os.path.exists(trips_file):
    sep = detect_separator(trips_file)
    with open(trips_file, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f, delimiter=sep)
        for row in reader:
            if row["service_id"] in active_service_ids:
                active_trip_ids.add(row["trip_id"])

print(f"Trips actifs trouvés: {len(active_trip_ids)}")

# 3. ÉTAPE 1 : Construire trip.stopsInOrder et trip.stopDetails (comme dans Java)
# Simuler la phase 4 de computeDepartureInfo (remplissage séquentiel)
trip_stops_in_order = {}  # trip_id -> [stop_id1, stop_id2, ...]
trip_stop_details = {}    # trip_id -> [{departureTime: "...", ...}, ...]

if os.path.exists(stop_times_file):
    sep = detect_separator(stop_times_file)
    with open(stop_times_file, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f, delimiter=sep)
        for row in reader:
            trip_id = row["trip_id"]
            if trip_id not in active_trip_ids:
                continue
            
            stop_id = row["stop_id"]
            departure_time = row["departure_time"]
            
            # Simuler trip.addStop(stopId) et trip.addStopDetail(...)
            if trip_id not in trip_stops_in_order:
                trip_stops_in_order[trip_id] = []
                trip_stop_details[trip_id] = []
            
            trip_stops_in_order[trip_id].append(stop_id)
            trip_stop_details[trip_id].append({"departureTime": departure_time})

# 4. ÉTAPE 2 : Créer departureTripsInfo (comme étape 5 de computeDepartureInfo)
departure_trips_info = {}  # trip_id -> [(stop_id, departure_in_seconds), ...]

for trip_id in active_trip_ids:
    if trip_id not in trip_stops_in_order:
        continue
    
    stops_in_order = trip_stops_in_order[trip_id]
    stop_details = trip_stop_details[trip_id]
    
    if len(stops_in_order) != len(stop_details):
        continue
    
    stop_pairs = []
    for i in range(len(stops_in_order)):
        stop_id = stops_in_order[i]
        departure_time = stop_details[i]["departureTime"]
        departure_in_seconds = convert_time_to_seconds(departure_time)
        stop_pairs.append((stop_id, str(departure_in_seconds)))
    
    departure_trips_info[trip_id] = stop_pairs

print(f"departureTripsInfo créé pour {len(departure_trips_info)} trips")

# 5. ÉTAPE 3 : Identifier les stops avec stop_sequence = 1 (comme étape 6 de computeDepartureInfo)
trip_to_first_stop = {}      # trip_id -> stop_id (stop_sequence=1)
trip_to_first_stop_time = {} # trip_id -> departure_time_seconds

# Parcourir stop_times.txt pour identifier stop_sequence = 1
with open(stop_times_file, 'r', encoding='utf-8') as f:
    reader = csv.DictReader(f, delimiter=detect_separator(stop_times_file))
    for row in reader:
        trip_id = row["trip_id"]
        if trip_id not in active_trip_ids:
            continue
        
        stop_id = row["stop_id"]
        departure_time = row["departure_time"]
        try:
            stop_sequence = int(row["stop_sequence"])
        except:
            continue
        
        # Si c'est le premier stop de la séquence (stop_sequence = 1)
        if stop_sequence == 1:
            trip_to_first_stop[trip_id] = stop_id
            trip_to_first_stop_time[trip_id] = convert_time_to_seconds(departure_time)

print(f"Trips avec stop_sequence=1 identifiés: {len(trip_to_first_stop)}")

# 6. ÉTAPE 4 : Créer stopToTripIds avec logique de dédoublonnage (comme étape 6 suite)
stop_to_trip_ids = {}  # stop_id -> [trip_id1, trip_id2, ...]
seen_trip_signatures = set()

for trip_id in departure_trips_info.keys():
    stop_pairs = departure_trips_info[trip_id]
    if not stop_pairs:
        continue
    
    # Utiliser le stop avec stop_sequence = 1 si disponible
    if trip_id in trip_to_first_stop:
        first_stop_id = trip_to_first_stop[trip_id]
        departure_time = str(trip_to_first_stop_time[trip_id])
    else:
        # Fallback : utiliser le premier dans la liste
        first_stop_id = stop_pairs[0][0]
        departure_time = stop_pairs[0][1]
       
    
    # Créer la signature pour éviter les doublons (exactement comme dans Java)
    stop_sequence_str = ";".join(pair[0] for pair in stop_pairs)
    signature = f"{first_stop_id}_{departure_time}_{stop_sequence_str}"
    
    if signature in seen_trip_signatures:
        continue
    seen_trip_signatures.add(signature)
    
    if first_stop_id not in stop_to_trip_ids:
        stop_to_trip_ids[first_stop_id] = []
    stop_to_trip_ids[first_stop_id].append(trip_id)

# 7. ÉTAPE 5 : Simuler l'affectation dans chaque stop (comme étape 7 de computeDepartureInfo)
# stop.ensureDepartureTripsInfo() + stop.addStopPairs(tripId, pairs)

# Structure finale : stop_id -> {trip_id: [(stop_id, time), ...]}
stops_departure_trips_info = {}  # Simule stop.getDepartureTripsInfo()

for stop_id, trip_ids in stop_to_trip_ids.items():
    # Trier les trips par heure de départ (comme dans Java)
    trip_ids.sort(key=lambda tid: trip_to_first_stop_time.get(tid, int(departure_trips_info[tid][0][1]) if tid in departure_trips_info and departure_trips_info[tid] else 0))
    
    stops_departure_trips_info[stop_id] = {}
    
    for trip_id in trip_ids:
        if trip_id in departure_trips_info:
            pairs = departure_trips_info[trip_id]
            stops_departure_trips_info[stop_id][trip_id] = pairs

# 8. CALCULS FINAUX (exactement comme dans le modèle GAML)
nombre_stops_avec_departs = 0
nombre_total_trips = 0
nombre_total_agent_stops = 0

for stop_id, departure_stops_info in stops_departure_trips_info.items():
    if departure_stops_info and len(departure_stops_info) > 0:
        nombre_stops_avec_departs += 1
        
        # Parcourir chaque tripId dans departureStopsInfo
        for trip_id, paires in departure_stops_info.items():
            nombre_total_trips += 1
            
            # Compter le nombre de paires (agent stops) pour ce trip
            if paires:
                nombre_total_agent_stops += len(paires)

# NOUVEAUX INDICES AJOUTÉS
# Indice 4 : Nombre de stops de départ distincts (tous les stops identifiés comme départ)
nombre_stops_depart_distincts = len(stop_to_trip_ids)

# Indice 5 : Nombre total de stops associés aux trips actifs (avec doublons)
nombre_total_stops_trips_actifs = 0
for trip_id in active_trip_ids:
    if trip_id in trip_stops_in_order:
        nombre_total_stops_trips_actifs += len(trip_stops_in_order[trip_id])

# 9. AFFICHAGE DES RÉSULTATS (exactement comme dans GAML)
print(f"\n=== STATISTIQUES DEPARTURESTOPSINFO (PYTHON) ===")
print(f"1. Nombre de stops de départ (avec departureStopsInfo non null): {nombre_stops_avec_departs}")
print(f"2. Nombre total de trips dans tous les departureStopsInfo: {nombre_total_trips}")
print(f"3. Nombre total d'agent stops (paires stopID-heure): {nombre_total_agent_stops}")
print(f"=========================================")


# Afficher quelques détails pour validation
if nombre_stops_avec_departs > 0:
    print(f"\nPremiers stops de départ: {list(stops_departure_trips_info.keys())[:5]}")
    first_stop = list(stops_departure_trips_info.keys())[0]
    print(f"Exemple - Stop {first_stop}: {len(stops_departure_trips_info[first_stop])} trips")

Services actifs trouvés: 15
Trips actifs trouvés: 7783
departureTripsInfo créé pour 7783 trips
Trips avec stop_sequence=1 identifiés: 7708

=== STATISTIQUES DEPARTURESTOPSINFO (PYTHON) ===
1. Nombre de stops de départ (avec departureStopsInfo non null): 187
2. Nombre total de trips dans tous les departureStopsInfo: 7782
3. Nombre total d'agent stops (paires stopID-heure): 200306

Premiers stops de départ: ['GMAR3', 'HPLA1', 'BSYS1', 'HALA1', 'GSSU3']
Exemple - Stop GMAR3: 76 trips
